In [1]:
# Imports
from pathlib import Path

import pandas as pd
import torch
from neuralhydrology.nh_run import start_run, eval_run, finetune

In [ ]:
!cat finetune.yml

In [5]:
# Add the path to the pre-trained model to the finetune config
with open("finetune.yml", "a") as fp:
    fp.write(f"\nbase_run_dir: {run_dir.absolute()}")
    
# # Create a basin file with the basin we selected above
# with open("finetune_basin.txt", "w") as fp:
#     fp.write(basin)

In [ ]:
finetune(Path("finetune.yml"))

In [ ]:
eval_run(run_dir, period="test")

In [ ]:
finetune_dir = Path("runs/...") # fill in with your created output file
eval_run(finetune_dir, period="test")

Now let's look at the test period results of the pre-trained base model and the finetuned model for the basin that we chose above.

In [ ]:

import pickle
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import torch
from neuralhydrology.evaluation import metrics
from neuralhydrology.nh_run import start_run, eval_run
import matplotlib.dates as mdates
import os
import numpy as np
area = 111639970.52
run_dir = Path('runs/...') # fill in with your created output file
for i in range(1, 51):
    eval_run(epoch = i,run_dir=run_dir, period="test")

# Initialize variables to track the best NSE and the corresponding epoch
max_nse = -float('inf')  # Set to negative infinity initially
best_epoch = None

# Loop through epochs 1 to 50
for i in range(1, 51):  # Epochs 1 to 50
    # Format the folder name for the current epoch
    epoch_folder = finetune_dir/ "test"/ f"model_epoch{i:03d}"
    metrics_file = epoch_folder / "test_metrics.csv"

    # Check if the test_metrics.csv file exists
    if not metrics_file.exists():
        print(f"test_metrics.csv file not found for epoch {i}. Skipping...")
        continue

    # Load the CSV file into a DataFrame
    df = pd.read_csv(metrics_file)

    # Extract the NSE value from the dataframe (assuming NSE is in a column named 'NSE')
    if 'NSE' in df.columns:
        nse_value = df['NSE'].iloc[0]  
        print(f"Epoch {i}: NSE = {nse_value:.4f}")

        # Check if this is the highest NSE found so far
        if nse_value > max_nse:
            max_nse = nse_value
            best_epoch = i

# Output the best epoch and its NSE value
if best_epoch is not None:
    print(f"\nThe epoch with the highest NSE is Epoch {best_epoch} with an NSE value of {max_nse:.4f}")
else:
    print("No NSE values found.")


with open(finetune_dir / "test" / "model_epoch..." / "test_results.p", "rb") as fp: # fill in with your best epoch
    results = pickle.load(fp)
qobs = results['6']['1D']['xr']['streamflow_obs']*area/(1000*3600*24)
qsim = results['6']['1D']['xr']['streamflow_sim']*area/(1000*3600*24)

fig, ax = plt.subplots(figsize=(16,10))
ax.plot(qobs['date'], qobs)
ax.plot(qsim['date'], qsim)
ax.set_ylabel("Discharge (m³/s)")

    #data properties

values = metrics.calculate_all_metrics(qobs.isel(time_step=-1), qsim.isel(time_step=-1))
for key, val in values.items():
    print(f"{key}: {val:.3f}")

In [ ]:
plt.figure(figsize=(6.3, 3.5))

# Plot observed streamflow
plt.plot(qobs['date'], qobs, label='Observed Discharge', color='blue', linewidth=1)

# Plot simulated streamflow
plt.plot(qobs['date'], qsim, label='Simulated Discharge', color='red', linewidth = 1)

# Add axis labels and styling
plt.xlabel('Date', fontsize=12)
plt.ylabel('Discharge (m³/s)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.xticks(rotation=45)
plt.style.use('seaborn-v0_8-whitegrid')

# Add legend
plt.legend(frameon=True, edgecolor='black')
plt.savefig("Final_output_finetuned_single.png", dpi=1000, bbox_inches='tight') 
# Show plot
plt.show()

In [ ]:
print(qobs)
print(qsim)
qsim_avg = qsim.mean(skipna = True)
qobs_avg = qobs.mean(skipna=True)
print(qsim_avg)
print(qobs_avg)
qsim_avg.values/qobs_avg.values

In [15]:
def NSE_LF(Qmod, Qobs):
    """
    Calculate the low-flow Nash-Sutcliffe Efficiency (NSE_LF) using inverse flows.
    This variant of NSE emphasizes performance during low-flow periods.

    Only calculates for timestamps where no NaN values are present in the observed flows.

    Parameters
    ----------
    Qmod: numpy.array
        Modelled flows
    Qobs: numpy.array
        Observed flows

    Returns
    -------
    nse_lf: float
        Low-flow Nash-Sutcliffe Efficiency

    """
    nan_bool = np.isnan(Qobs)
    Qmod_nonan = Qmod[~nan_bool]
    Qobs_nonan = Qobs[~nan_bool]

    # Avoid division by zero using epsilon (1/100th of mean observed flow)
    epsilon = np.mean(Qobs_nonan) / 100

    Qmod_inv = 1 / (Qmod_nonan + epsilon)
    Qobs_inv = 1 / (Qobs_nonan + epsilon)

    T = (Qobs_inv - Qmod_inv) ** 2
    N = (Qobs_inv - np.mean(Qobs_inv)) ** 2

    nse_lf = 1 - np.sum(T) / np.sum(N)
    return nse_lf

In [ ]:
NSE_LF(qsim.values, qobs.values)